In [5]:
#load aiida environment and connect to database
from aiida import load_profile
profile = load_profile()

# load classes and functions
from aiida.orm import CifData, Code, Dict, load_node
from aiida.engine import submit
from aiida_kkr.tools import kkrparams, plot_kkr

In [6]:
metadata_option_1 = {'max_wallclock_seconds': 36000,'resources': 
               {'tot_num_mpiprocs': 1, 'num_machines': 1},
              'custom_scheduler_commands': 
              '#SBATCH --account=jara0191\n\nulimit -s unlimited; export OMP_STACKSIZE=2g',
              'withmpi':
              True}

In [7]:
from aiida.plugins import DataFactory
from aiida.orm import Dict, StructureData, Code, CifData, load_code, load_node

In [26]:
alat = 3.61 #in Ang.
bravais = [[0.5*alat, 0.5*alat, 0], [0.5*alat, 0, 0.5*alat], [0, 0.5*alat, 0.5*alat]]

Cu = StructureData(cell=bravais)
Cu.append_atom(position=[0,0,0], symbols= 'Cu')

In [27]:
from masci_tools.io.kkr_params import kkrparams
params = kkrparams(params_type='voronoi')


In [28]:
params_dict = {'LMAX':2, 'NSPIN':1, 'RCLUSTZ':2.3}
params.set_multiple_values(**params_dict)
voro_param_node = Dict(dict=params.get_dict())

In [29]:
params.get_missing_keys()

['ALATBASIS', 'BRAVAIS', 'NAEZ', '<RBASIS>', '<ZATOM>']

In [30]:
code = Code.get_from_string('voro@claix18_init')
builder = code.get_builder()

In [31]:
builder.metadata.options= metadata_option_1
builder.structure = Cu
builder.metadata.label = 'voronoi calc for Cu'
builder.parameters = voro_param_node

In [32]:
params.get_missing_keys()

['ALATBASIS', 'BRAVAIS', 'NAEZ', '<RBASIS>', '<ZATOM>']

In [36]:
# voro_calc = submit(builder) # uuid : 'cc9b0ea1-95f4-4e84-8396-e9463272fd00'



In [37]:
voro_calc = load_node('cc9b0ea1-95f4-4e84-8396-e9463272fd00')

voro_remote_data = voro_calc.outputs.remote_folder
voro_params_node =voro_calc.inputs.parameters
voro_params_dict = voro_params_node.get_dict()


In [38]:
params_kkr = kkrparams(params_type = 'kkr', **voro_params_dict) 
params_kkr.set_multiple_values(
                    NSTEPS=100, EMIN= -0.4, EMAX= 1.0,
                    RMAX=7., TEMPR=600.0,
                    GMAX=65., NPT1=3, NPOL=6, NPT2=12, NPT3=3,
                    IMIX=0,
                    STRMIX=0.05,
                    QBOUND=1e-2)

In [39]:
code = Code.get_from_string('kkr@claix18_init')
builder_kkr = code.get_builder()
builder_kkr.parameters = Dict(dict=params_kkr.get_dict())
builder_kkr.metadata.options = metadata_option_1
builder_kkr.parent_folder = voro_remote_data


In [20]:
kkr_calc = submit(builder_kkr)
#  uuid '2326ffc6-f52b-456b-8554-64041d682dba'

In [41]:
kkr_calc = load_node('2326ffc6-f52b-456b-8554-64041d682dba' )
kkr_remote_data = kkr_calc.outputs.remote_folder
code = kkr_calc.inputs.code
params_kkr = kkrparams(params_type = 'kkr',
                       **kkr_calc.inputs.parameters.get_dict())
params_kkr.set_multiple_values(NSTEPS=200, QBOUND=1e-8, IMIX=5)

In [42]:
# (NSTEPS=200, IMIX=5, QBOUND=1.0E-8)
QBOUND=1.0E-8
QBOUND


1e-08

In [43]:
builder_kkr = code.get_builder()
builder_kkr.parameters = Dict(dict=params_kkr.get_dict())
builder_kkr.metadata.options= metadata_option_1
builder_kkr.parent_folder= kkr_remote_data


In [45]:
# kkr_calc_continued = submit(builder_kkr)

# uuid '1c97655a-21ea-42e5-b799-e8c76de15471'
# kkr_calc_continued.uuid

In [46]:
kkr_calc = load_node('1c97655a-21ea-42e5-b799-e8c76de15471')


In [3]:
#### The impurity steps are here ####
## Writeout Green Function and tmat for host here Cu ##

In [47]:
# converge_kkr= load_node('1c97655a-21ea-42e5-b799-e8c76de15471')
kkr_converged_parent_folder = kkr_calc.outputs.remote_folder
converge_kkr  = kkr_converged_parent_folder.get_incoming().first().node


In [48]:

converge_kkr_dict = converge_kkr.inputs.parameters.get_dict()
converge_kkr_dict['RUNOPT'] = ['KKRFLEX']
converge_kkr_dict['NSTEPS'] = 1

In [49]:
imp_info = Dict(dict={'Rcut':4.0, 'ilayer_center': 0, 'Zimp':[79.]})

In [227]:
# converge_kkr_dict

'Au'

In [51]:
##
code = Code.get_from_string('kkrflex@claix18_init')
code = converge_kkr.inputs.code
builder_kkr_gf = code.get_builder()
builder_kkr_gf.parameters = Dict(dict=converge_kkr_dict)
builder_kkr_gf.metadata.options= metadata_option_1
builder_kkr_gf.parent_folder= converge_kkr.outputs.remote_folder
builder_kkr_gf.impurity_info = imp_info

In [52]:
kkr_host_gf = submit(builder_kkr_gf)
#'b260a943-a0a0-4cbc-b17f-f41168a74c99' and pk=2988

In [135]:
voro_calc = load_node('cc9b0ea1-95f4-4e84-8396-e9463272fd00')
structure = voro_calc.inputs.structure

from aiida.engine import workfunction, calcfunction
@calcfunction
def change_struc_imp_aux_wf(struc, imp_info):
    struc = structure
    from aiida.common.constants import elements
    _atomic_numbers = {data['symbol']:num for num, data in elements.items()}
    new_imp_struc = StructureData(cell=struc.cell)

    for site in struc.sites:
        kname = site.kind_name
        pos = site.position
        kind = struc.get_kind(kname)
        zatom = _atomic_numbers[kind.get_symbols_string()]
        isite =0
        imp_dict = imp_info.get_dict()
        if isite == imp_dict.get('ilayer_center'):
            zatom = imp_dict.get('Zimp')[0]
        symbol = elements.get(zatom).get('symbol')    
        new_imp_struc.append_atom(position=pos, symbols=symbol)
        isite += 1       
    return new_imp_struc
# new_struc = change_struc_imp_aux_wf(structure, imp_info)
# new_struc # uuid '58a2f6ea-6381-41da-9c4d-8910752fc8aa'


In [171]:
new_struc = load_node('58a2f6ea-6381-41da-9c4d-8910752fc8aa')
kkrcalc_converged = load_node('1c97655a-21ea-42e5-b799-e8c76de15471')
codename = 'voro@claix18_init'
code = Code.get_from_string(codename)

builder = code.get_builder()
builder.metadata.options = metadata_option_1
builder.structure = new_struc
builder.parameters = kkrcalc_converged.inputs.parameters

# voro_calc_aux = submit(builder)
# voro_calc_aux # uuid 'fb8fff67-0c29-40f9-96b0-63497f9ae955' pk =3016


In [190]:
# kkrcakkrcalc_converged pk=2924
kkrcalc_converged = load_node(2924)
GF_host_calc = load_node(2988)
voro_calc_aux = load_node(3016)

In [191]:
from aiida_kkr.tools.common_workfunctions import neworder_potential_wf
from numpy import loadtxt

potname_converged = None
potname_imp = 'potential_imp'
file_opened=GF_host_calc.outputs.retrieved.open('scoef')
neworder_pot1 = [int(i) for i in loadtxt(file_opened, skiprows=1)[:,3]-1]

potname_impvorostart = None
replacelist_pot2 = [[0,0]]

settings_dict = {'pot1': potname_converged,  'out_pot': potname_imp, 'neworder': neworder_pot1,
                 'pot2': potname_impvorostart, 'replace_newpos': replacelist_pot2, 'label': 'startpot_KKRimp',
                 'description': 'starting potential for Au impurity in bulk Cu'}
settings = Dict(dict=settings_dict)

startpot_Au_imp_sfd = neworder_potential_wf(settings_node=settings,
                                            parent_calc_folder=kkrcalc_converged.outputs.remote_folder,
                                            parent_calc_folder2=voro_calc_aux.outputs.remote_folder)


/opt/aiida-core/aiida/orm/nodes/node.py:63: AiidaDeprecationWarning: 
The method `del` was called on the return value of `<FolderData: uuid: a3630a7c-becb-431b-a311-1059f01ecac7 (pk: 2990)>.open()` outside of a context manager.
Please wrap this call inside `with <node instance>.open(): ...` to silence this warning. This will raise an exception, starting from `aiida-core==2.0.0`.
The offending call comes from:
  File "<ipython-input-191-115eaf414f1d>", line 6, in <module>
    file_opened=GF_host_calc.outputs.retrieved.open('scoef')

  warnings.warn(msg, AiidaDeprecationWarning)  # pylint: disable=no-member
02/04/2021 10:57:24 AM <20087> aiida.orm.nodes.process.calculation.calcfunction.CalcFunctionNode: [REPORT] [3023|neworder_potential_wf|on_except]: Traceback (most recent call last):
  File "/opt/aiida-kkr/aiida_kkr/tools/common_workfunctions.py", line 918, in neworder_potential_wf
    pot2 = extract_potname_from_remote(parent_calc_folder2)
  File "/opt/aiida-kkr/aiida_kkr/tools/common

InputValidationError: settings_node_dict needs to have key "pot2" containing the filename of the input potential

In [187]:
GF_calc_retr_list =GF_host_calc.outputs.retrieved.list_object_names()
if 'scoef' in GF_calc_retr_list:
    print('scoef')
    file_opened=GF_host_calc.outputs.retrieved.open('scoef')
    neworder_pot1 = [int(i) for i in loadtxt(file_opened, skiprows=1)[:,3]-1]


scoef


In [188]:
neworder_pot1

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
converge_kkr.get_builder_restart()